from google.colab import drive
drive.mount("/content/gdrive/")
%cd /content/gdrive/My\ Drive/work_space/Chatbot/NLU/Mark1
%pip install import_ipynb
import import_ipynb

In [3]:
%pip install persian
%pip install hazm
import hazm
import persian
import os
import re
import pandas as pd
import DataSet
import numpy as np
from hazm import *
if os.path.exists('../resources/postagger.model'):
  tagger = POSTagger(model='../resources/postagger.model')
else:
  raise ValueError("Wrong path is POSTagger directory")
stemmer = Stemmer()
lemmatizer = Lemmatizer()
from nltk import ngrams
vocabs,IDF = 0,0

     |████████████████████████████████| 317kB 5.1MB/s 
     |████████████████████████████████| 1.4MB 19.9MB/s 
     |████████████████████████████████| 235kB 51kB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394468 sha256=cb4f70573e537a288af0313d1f3e35bc94d85f31c0f65e706b5a6559a1535d04
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154141 sha256=844870563b14e45d11f7e67a8c54bfa679c5e9d5fb883a84856722a32f03cf57
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
importing Jupyter notebook from DataSet.ipynb


In [0]:
def normalize(text):
  #Basic ///
  norm = Normalizer()
  text = persian.convert_ar_characters(text)
  text = persian.convert_ar_numbers(text)
  text = text.strip()
  text = norm.normalize(persian.convert_en_numbers(text))

  #Advanced
  sent_past = []
  sent_future=[]

  pos = tagger.tag(word_tokenize(text))
  for word,p in pos :
    if "V" != p[0]:
      w = stemmer.stem(word) if len(stemmer.stem(word).strip()) > 1 else ""
      sent_past.append(w)
      sent_future.append(w)
      

    else :
        lem_res = lemmatizer.lemmatize(word).split("#")
        sent_past.append(lem_res[0])
        try:
          sent_future.append(lem_res[1])
        except:
          sent_future.append(lem_res[0])

  sent_past=  " ".join(sent_past)
  sent_future= " ".join(sent_future)
  res = [sent_past,sent_future] if sent_past != sent_future else [sent_past]
  return res

# normalize("می‌خوام .")

In [98]:
[1,2,3,4][:2]

[1, 2]

In [0]:
def __to_normilize(row,**kywrd):
  res = normalize(row["text"])
  if len(res) > 1:
    row["sent_future"] = res[1]
    row["sent_past"] = res[0]
  else:
    row["sent_past"] = res[0]
  return row

def __to_category(row,**kywd):
  categories = kywd["categories"]
  
  label = categories.index(row["intent"])
  return label

def __docs_prepare(docs):
  temp = docs.apply(__to_normilize,axis=1)
  f = temp[["sent_future","intent"]]
  p =temp[["sent_past","intent"]]
  f.columns=["text","intent"]
  p.columns=["text","intent"]
  res = f.append(p).dropna()

  classes = list(set(res["intent"]))
  labels = res.apply(__to_category,categories=classes,axis=1)
  return res,labels,classes




In [0]:

def __to_ngram(row,**kywrd):
  intent_ngram = kywrd["intent_ngram"]
  text =intent_ngram[intent_ngram["intent"] == row["intent"]]["text"].values[0]
   
  ntext = normalize(re.sub("\(("+row["name"]+")\)","%%%%",text ))
  # if rowprint(text)
  ngrams_pattern = ngrams(word_tokenize(ntext[1]),3),ngrams(word_tokenize(ntext[0]),3)
  grams_df = []

  for ngram in ngrams_pattern:
    for gram in ngram:
      if "٪٪٪٪" in gram[:2]:
        grams_df.append(" ".join(gram))
  row["ngrams"] = grams_df
  return row


docs,entities,slots,intent_ngram = DataSet.loader(lang = "en" ,path= "../dataset")

for i,row in slots.iterrows()[1]:
row=slots.iloc[4]
text =intent_ngram[intent_ngram["intent"] == row["intent"]]["text"].values[0]
  
ntext = normalize(re.sub("\(("+row["name"]+")\)","٪٪٪٪",text ))
# if rowprint(text)
ngrams_pattern = ngrams(word_tokenize(ntext[1]),3),ngrams(word_tokenize(ntext[0]),3)
grams_df = []

for ngram in ngrams_pattern:
  for gram in ngram:
    if "٪٪٪٪" in gram[:1]:
      grams_df.append( " ".join(gram))
row["ngrams"] = grams_df
grams_df[-1]

In [0]:
def prepare_dataset(docs,slots,intent_ngram):
  docs,labels,classes= __docs_prepare(docs)
  slots = slots.apply(__to_ngram,intent_ngram=intent_ngram,axis=1)

  return docs,labels,classes,slots
docs,entities,slots,intent_ngram = DataSet.loader(lang = "en" ,path= "../dataset")  
docs,labels,classes,slots = prepare_dataset(docs,slots,intent_ngram)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
def docs2vectors (docs):
  global vocabs
  global IDF
  vocabs = " ".join(docs).split(" ")
  vocabs = list(set([wrd.strip() for wrd in vocabs]))[1:]

  tf_idf = TfidfVectorizer(vocabulary=vocabs).fit(docs)
  IDF = tf_idf.idf_
  vocabs = tf_idf.get_feature_names()
  tf_idf = pd.DataFrame(tf_idf.transform(docs).toarray())
  
  return tf_idf

# tf_idf = docs2vectors(docs=docs["text"])


In [9]:
%pip install jellyfish
import jellyfish


     |████████████████████████████████| 143kB 4.5MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=73006 sha256=d4058e6bfaa3d32da40b743951b386d5252327cf797770b3647048ba3d13e86c
  Stored in directory: /root/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
Successfully built jellyfish


In [0]:

def __distanceـdf(row , **kywrd):
  target = kywrd["target"]
  col_name = kywrd["col_name"]
  param =row[col_name] , target
  
  levenshtein = 1-(jellyfish.levenshtein_distance(*param)*2)/ (len(row[col_name])+len(target))
  hamming = 1-(jellyfish.hamming_distance(*param)*2)/ (len(row[col_name])+len(target))

  row["levenshtein"],row["hamming"]= levenshtein,hamming
  return row

def similarity(a ,b):
  param =a , b
  levenshtein = 1-(jellyfish.levenshtein_distance(*param)*2)/ (len(param[0])+len(param[1]))
  hamming = 1-(jellyfish.hamming_distance(*param)*2)/ (len(param[0])+len(param[1]))
  return levenshtein,hamming

def Dictation_correction(target):
  pos = tagger.tag([target])
  if (pos[0][1][0] =="V"):
    if target.startswith("می‌"):
      target = target[3:] 
    elif target.startswith("می"):
      target = target[2:] 

  target = target.strip()
  vocabs_df = pd.DataFrame(columns=["vocabs"])
  vocabs_df["vocabs"] = vocabs
  search_res = vocabs_df.apply(__distanceـdf,target= target,col_name="vocabs",axis=1)
  arg = search_res.iloc[np.argmax(search_res["levenshtein"])]
  
  if arg["hamming"] > 0.65 and arg["levenshtein"] > 0.65  :
    # print(arg)
    return arg["vocabs"] 
  else:
    
    return 0

# Dictation_correction(target="خواسم")

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
def query2vector(query):
  global IDF
  global vocabs
  
  n_text = []
  for word in word_tokenize(Normalizer().normalize(query)):
    if len(word) > 1:
      if tagger.tag([word])[0][1][0] != "V": 
        st = Dictation_correction(word)
        if st != 0:
          n_text.append(st)
        else:
          continue
      else:
        try:
          n_text.append(lemmatizer.lemmatize(word).split("#")[np.random.randint(0,2)])
        except:
          n_text.append(lemmatizer.lemmatize("میخام"))

  n_text = " ".join(n_text)
  # print(n_text)
  tf = CountVectorizer(vocabulary=vocabs).fit_transform(pd.Series({"query":n_text}))
  Qtfidf = tf.toarray()*IDF

  return Qtfidf

# query = " میخوام درس هوش محاسباتی را اخذ کنم و بینایی را حذف کنم ."
# query2vector(query)